In [1]:
import mxfold2
import torch
import numpy as np
from mxfold2.predict import Predict
from tqdm import tqdm

In [2]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
config = { 'max_helix_length': 30,
            'embed_size' : 64,
            'num_filters': [64,64,64,64,64,64,64,64],
            'filter_size': [5,3,5,3,5,3,5,3],
            'pool_size': [1],
            'dilation':0, 
            'num_lstm_layers': 2, 
            'num_lstm_units': 32,
            'num_transformer_layers': 0,
            'num_transformer_hidden_units': 2048,
            'num_transformer_att': 8,
            'num_hidden_units': [32],
            'num_paired_filters':[64,64,64,64,64,64,64,64],
            'paired_filter_size': [5,3,5,3,5,3,5,3],
            'dropout_rate': 0.5,
            'fc_dropout_rate': 0.5,
            'num_att': 8,
            'pair_join': 'cat',
            'no_split_lr': None
        }

In [3]:
args = Namespace(model='MixC',**config)
test = Predict()
test.model,config = test.build_model(args)
test.model.load_state_dict(torch.load('./mxfold2/testfold2/models/TrainSetAB.pth'))

<All keys matched successfully>

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [15]:
test.model.cpu()

MixedFold(
  (turner): RNAFold()
  (zuker): ZukerFold(
    (net): NeuralNet(
      (embedding): SparseEmbedding(
        (embedding): Embedding(6, 64, padding_idx=0)
      )
      (encoder): CNNLSTMEncoder(
        (dropout): Dropout(p=0.5, inplace=False)
        (conv): CNNLayer(
          (net): ModuleList(
            (0): Sequential(
              (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
              (1): Identity()
              (2): GroupNorm(1, 64, eps=1e-05, affine=True)
              (3): CELU(alpha=1.0)
              (4): Dropout(p=0.5, inplace=False)
            )
            (1): Sequential(
              (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
              (1): Identity()
              (2): GroupNorm(1, 64, eps=1e-05, affine=True)
              (3): CELU(alpha=1.0)
              (4): Dropout(p=0.5, inplace=False)
            )
            (2): Sequential(
              (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), pad

In [18]:
test.model.cuda() # .to("cuda:0") .to(device)

MixedFold(
  (turner): RNAFold()
  (zuker): ZukerFold(
    (net): NeuralNet(
      (embedding): SparseEmbedding(
        (embedding): Embedding(6, 64, padding_idx=0)
      )
      (encoder): CNNLSTMEncoder(
        (dropout): Dropout(p=0.5, inplace=False)
        (conv): CNNLayer(
          (net): ModuleList(
            (0): Sequential(
              (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), padding=(2,))
              (1): Identity()
              (2): GroupNorm(1, 64, eps=1e-05, affine=True)
              (3): CELU(alpha=1.0)
              (4): Dropout(p=0.5, inplace=False)
            )
            (1): Sequential(
              (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
              (1): Identity()
              (2): GroupNorm(1, 64, eps=1e-05, affine=True)
              (3): CELU(alpha=1.0)
              (4): Dropout(p=0.5, inplace=False)
            )
            (2): Sequential(
              (0): Conv1d(64, 64, kernel_size=(5,), stride=(1,), pad

In [32]:
test.model.cuda();

In [39]:
%%time
with torch.no_grad():
    test.model.forward(['ATACCCCC'])

CPU times: user 7.92 ms, sys: 3.91 ms, total: 11.8 ms
Wall time: 11 ms


In [27]:
test.model.cpu();

In [31]:
%%time
with torch.no_grad():
    test.model.forward(['ATACCCCC'])

CPU times: user 25 s, sys: 9.08 ms, total: 25 s
Wall time: 1.43 s


In [43]:
file1 = open('./data/annotation/seq.txt', 'r')
Lines = file1.readlines()

In [44]:
import math
def reformat_seq(X0,SL):
    num_points = int(math.ceil(len(X0)/ SL))
    Xd = []
    X0 = np.pad(X0, [0, SL], 'constant', constant_values='N')
    for i in range(num_points):
        Xd.append(''.join(X0[SL*i:SL*(i+1)]))
    return Xd

In [ ]:
torch.

In [46]:
%%time
for line in tqdm(Lines):
    line_seq = line.split()[-1]
    seq_list = reformat_seq(list(line_seq),2000)
    with torch.set_grad_enabled(False
                               ):
        for seq in seq_list:
            for i in range(10):
                scs,preds,bps = test.model.forward([seq[i*200:(i+1)*200]],
                                            return_partfunc=False)

  0%|                                    | 9/61507 [00:55<105:55:49,  6.20s/it]


KeyboardInterrupt: 